In [1]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from matplotlib import pyplot as plt



# 1. Build and train a network to recognize digits


## A. Get the MNIST digit data set
The MNIST digit data consists of a training set of 60k 28x28 labeled digits and a test set of 10k 28x28 labeled digits. The data set can be imported directly from the torchvision package as torchvision.datasets.MNIST. Use the matplotlib pyplot package or OpenCV to look at the first six example digits. Look up examples that make use of the pyplot subplot method to create a grid of plots.

Include a plot of the first six example digits in your report.


In [2]:


def loadData():
    batch_train= 1000
    data_loader = torch.utils.data.DataLoader(
        torchvision.datasets.MNIST(
            'mnist',
            train=True,
            download=True,
            transform=torchvision.transforms.Compose(
                [torchvision.transforms.ToTensor(),
                 torchvision.transforms.Normalize(
                    #   normalize with mean and std
                    (0.1307,), (0.3801,)
                )
                ])),
        batch_size=batch_train,
        shuffle=False)


    
    # Enumerate on test loader will give me batch index 
    # the train loader itself returning the images(example_data) and the label for it (example_target)
    return data_loader



    

In [3]:
def plotData(data_loader) :
    figure = plt.figure(figsize=(8, 8))
    
    # get the X and y
    examples = enumerate(data_loader)
    batch_idx, (sample_data, sample_targets) = next(examples)
    print("examining train_data")
    
    # print
    print(batch_idx)
    print(sample_data.shape)
    print(sample_targets.shape)
    
    
    # plot the first 6 sample
    fig = plt.figure()
    for i in range(6):
        plt.subplot(2, 3, i+1)
        plt.tight_layout()
        plt.imshow(training_data[i][0], cmap="gray", interpolation="none")
        plt.title("Ground truth: %d" %(sample_targets[i]))
        plt.xticks([])
                  
    plt.show()

In [6]:
data_loader = loadData()
plot_data(data_loader)


NameError: name 'load_data' is not defined